<a href="https://colab.research.google.com/github/rishita3863/Performance-comparision-IaaS-and-PaaS/blob/main/Copy_of_CC_Internal_Project_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install psutil requests


In [2]:
import requests
import time
import psutil


In [3]:
IAAS_URL = "https://httpbin.org/get"
PAAS_URL = "https://jsonplaceholder.typicode.com/posts/1"

In [4]:
def benchmark_service(url, num_requests=10):
    response_times = []
    cpu_usages = []
    memory_usages = []

    for i in range(num_requests):
        # CPU and memory before request
        cpu_before = psutil.cpu_percent(interval=None)
        mem_before = psutil.virtual_memory().percent

        # Send request and measure response time
        start_time = time.time()
        response = requests.get(url)
        end_time = time.time()

        # CPU and memory after request
        cpu_after = psutil.cpu_percent(interval=None)
        mem_after = psutil.virtual_memory().percent

        # Calculate response time in ms
        resp_time = (end_time - start_time) * 1000

        # Save values
        response_times.append(resp_time)
        cpu_usages.append((cpu_before + cpu_after) / 2)
        memory_usages.append((mem_before + mem_after) / 2)

        print(f"Request {i+1}: {resp_time:.2f} ms | CPU {cpu_usages[-1]:.2f}% | Memory {memory_usages[-1]:.2f}%")

    # Average values
    avg_response = sum(response_times) / len(response_times)
    avg_cpu = sum(cpu_usages) / len(cpu_usages)
    avg_memory = sum(memory_usages) / len(memory_usages)

    return avg_response, avg_cpu, avg_memory


In [5]:
print("Benchmarking IaaS...")
iaas_results = benchmark_service(IAAS_URL, num_requests=10)

print("\nBenchmarking PaaS...")
paas_results = benchmark_service(PAAS_URL, num_requests=10)


Benchmarking IaaS...
Request 1: 375.98 ms | CPU 86.85% | Memory 8.00%
Request 2: 655.67 ms | CPU 18.20% | Memory 8.15%
Request 3: 3873.11 ms | CPU 38.15% | Memory 8.75%
Request 4: 361.99 ms | CPU 85.40% | Memory 9.35%
Request 5: 1004.33 ms | CPU 41.80% | Memory 9.50%
Request 6: 258.64 ms | CPU 43.25% | Memory 9.65%
Request 7: 499.19 ms | CPU 41.50% | Memory 9.80%
Request 8: 279.61 ms | CPU 97.25% | Memory 9.95%
Request 9: 660.87 ms | CPU 94.30% | Memory 10.15%
Request 10: 1032.92 ms | CPU 38.80% | Memory 10.45%

Benchmarking PaaS...
Request 1: 46.89 ms | CPU 4.55% | Memory 10.60%
Request 2: 30.96 ms | CPU 14.30% | Memory 10.60%
Request 3: 36.52 ms | CPU 6.25% | Memory 10.60%
Request 4: 34.53 ms | CPU 0.00% | Memory 10.60%
Request 5: 36.61 ms | CPU 6.25% | Memory 10.60%
Request 6: 31.26 ms | CPU 0.00% | Memory 10.60%
Request 7: 29.48 ms | CPU 8.35% | Memory 10.60%
Request 8: 35.79 ms | CPU 0.00% | Memory 10.60%
Request 9: 29.83 ms | CPU 8.35% | Memory 10.60%
Request 10: 36.06 ms | CPU 0

In [6]:
print("\n=== Final Comparison ===")
print(f"IaaS -> Response: {iaas_results[0]:.2f} ms | CPU: {iaas_results[1]:.2f}% | Memory: {iaas_results[2]:.2f}%")
print(f"PaaS -> Response: {paas_results[0]:.2f} ms | CPU: {paas_results[1]:.2f}% | Memory: {paas_results[2]:.2f}%")



=== Final Comparison ===
IaaS -> Response: 900.23 ms | CPU: 58.55% | Memory: 9.38%
PaaS -> Response: 34.79 ms | CPU: 4.80% | Memory: 10.60%


In [7]:
!pip install plotly


In [8]:
results = {
    "IaaS": {
        "Response Time (ms)": iaas_results[0],
        "CPU Usage (%)": iaas_results[1],
        "Memory Usage (%)": iaas_results[2]
    },
    "PaaS": {
        "Response Time (ms)": paas_results[0],
        "CPU Usage (%)": paas_results[1],
        "Memory Usage (%)": paas_results[2]
    }
}


In [9]:
import plotly.graph_objects as go

# Prepare data
categories = ["Response Time (ms)", "CPU Usage (%)", "Memory Usage (%)"]
iaas_values = [results["IaaS"][cat] for cat in categories]
paas_values = [results["PaaS"][cat] for cat in categories]

# Create bar chart
fig = go.Figure(data=[
    go.Bar(name="IaaS", x=categories, y=iaas_values, marker_color="blue"),
    go.Bar(name="PaaS", x=categories, y=paas_values, marker_color="green")
])

# Update layout
fig.update_layout(
    title="IaaS vs PaaS Performance Comparison",
    xaxis_title="Metrics",
    yaxis_title="Values",
    barmode="group",
    template="plotly_white"
)

# Show in Colab
fig.show()


In [10]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Bar Comparison", "Results Table"),
    specs=[[{"type": "bar"}],
           [{"type": "table"}]]
)

# Add bar chart
fig.add_trace(go.Bar(name="IaaS", x=categories, y=iaas_values, marker_color="blue"), row=1, col=1)
fig.add_trace(go.Bar(name="PaaS", x=categories, y=paas_values, marker_color="green"), row=1, col=1)

# Add table
fig.add_trace(go.Table(
    header=dict(values=["Metric", "IaaS", "PaaS"], fill_color="lightgrey", align="center"),
    cells=dict(values=[categories, iaas_values, paas_values], align="center")
), row=2, col=1)

# Layout
fig.update_layout(title="IaaS vs PaaS Dashboard", barmode="group", template="plotly_white")
fig.show()


In [11]:
pip install gradio requests psutil plotly pandas

In [12]:
import gradio as gr
import requests
import time
import psutil
import plotly.graph_objects as go
import pandas as pd

def benchmark_service(url, num_requests=10):
    response_times = []
    cpu_usages = []
    memory_usages = []

    for i in range(num_requests):
        # CPU and memory before request
        cpu_before = psutil.cpu_percent(interval=None)
        mem_before = psutil.virtual_memory().percent

        # Send request and measure response time
        start_time = time.time()
        response = requests.get(url)
        end_time = time.time()

        # CPU and memory after request
        cpu_after = psutil.cpu_percent(interval=None)
        mem_after = psutil.virtual_memory().percent

        # Calculate response time in ms
        resp_time = (end_time - start_time) * 1000

        # Save values
        response_times.append(resp_time)
        cpu_usages.append((cpu_before + cpu_after) / 2)
        memory_usages.append((mem_before + mem_after) / 2)

        print(f"Request {i+1}: {resp_time:.2f} ms | CPU {cpu_usages[-1]:.2f}% | Memory {memory_usages[-1]:.2f}%")

    # Average values
    avg_response = sum(response_times) / len(response_times)
    avg_cpu = sum(cpu_usages) / len(cpu_usages)
    avg_memory = sum(memory_usages) / len(memory_usages)

    return avg_response, avg_cpu, avg_memory

def benchmark_compare(iaas_url, paas_url):
    print("Benchmarking IaaS...")
    iaas_results = benchmark_service(iaas_url, num_requests=10)

    print("\nBenchmarking PaaS...")
    paas_results = benchmark_service(paas_url, num_requests=10)

    results = {
        "IaaS": {
            "Response Time (ms)": iaas_results[0],
            "CPU Usage (%)": iaas_results[1],
            "Memory Usage (%)": iaas_results[2]
        },
        "PaaS": {
            "Response Time (ms)": paas_results[0],
            "CPU Usage (%)": paas_results[1],
            "Memory Usage (%)": paas_results[2]
        }
    }

    # Create DataFrame for table
    df = pd.DataFrame(results).round(2)

    # Create bar chart
    categories = ["Response Time (ms)", "CPU Usage (%)", "Memory Usage (%)"]
    iaas_values = [results["IaaS"][cat] for cat in categories]
    paas_values = [results["PaaS"][cat] for cat in categories]

    fig = go.Figure(data=[
        go.Bar(name="IaaS", x=categories, y=iaas_values, marker_color="blue"),
        go.Bar(name="PaaS", x=categories, y=paas_values, marker_color="green")
    ])

    fig.update_layout(
        title="IaaS vs PaaS Performance Comparison",
        xaxis_title="Metrics",
        yaxis_title="Values",
        barmode="group",
        template="plotly_white"
    )

    print("\n=== Final Comparison ===")
    print(f"IaaS -> Response: {iaas_results[0]:.2f} ms | CPU: {iaas_results[1]:.2f}% | Memory: {iaas_results[2]:.2f}%")
    print(f"PaaS -> Response: {paas_results[0]:.2f} ms | CPU: {paas_results[1]:.2f}% | Memory: {paas_results[2]:.2f}%")

    return fig, df

with gr.Blocks(title="IaaS vs PaaS Benchmark") as demo:
    gr.Markdown("# IaaS vs PaaS Performance Benchmark")
    gr.Markdown("Enter the URLs for IaaS and PaaS services to compare response time, CPU, and memory usage over 10 requests.")

    with gr.Row():
        iaas_input = gr.Textbox(
            label="IaaS URL",
            value="https://httpbin.org/get",
            placeholder="Enter IaaS service URL (e.g., https://httpbin.org/get)"
        )
        paas_input = gr.Textbox(
            label="PaaS URL",
            value="https://jsonplaceholder.typicode.com/posts/1",
            placeholder="Enter PaaS API URL (e.g., https://jsonplaceholder.typicode.com/posts/1)"
        )

    with gr.Row():
        submit_btn = gr.Button("Run Benchmark", variant="primary")

    with gr.Row():
        output_graph = gr.Plot(label="Performance Comparison Chart")

    with gr.Row():
        output_table = gr.Dataframe(label="Results Table", interactive=False)

    # Event handler
    submit_btn.click(
        fn=benchmark_compare,
        inputs=[iaas_input, paas_input],
        outputs=[output_graph, output_table]
    )

if __name__ == "__main__":
    demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8cff2919402e0dfe60.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Benchmarking IaaS...
Request 1: 3178.22 ms | CPU 21.60% | Memory 11.30%
Request 2: 614.68 ms | CPU 2.45% | Memory 11.30%
Request 3: 396.66 ms | CPU 1.25% | Memory 11.30%
Request 4: 817.36 ms | CPU 1.50% | Memory 11.30%
Request 5: 348.99 ms | CPU 2.90% | Memory 11.30%
Request 6: 1135.07 ms | CPU 1.10% | Memory 11.30%
Request 7: 922.70 ms | CPU 1.85% | Memory 11.30%
Request 8: 818.40 ms | CPU 0.30% | Memory 11.30%
Request 9: 1283.96 ms | CPU 1.55% | Memory 11.30%
Request 10: 362.06 ms | CPU 0.70% | Memory 11.30%

Benchmarking PaaS...
Request 1: 46.38 ms | CPU 5.55% | Memory 11.30%
Request 2: 33.61 ms | CPU 0.00% | Memory 11.30%
Request 3: 30.13 ms | CPU 7.15% | Memory 11.30%
Request 4: 32.48 ms | CPU 0.00% | Memory 11.30%
Request 5: 33.40 ms | CPU 7.15% | Memory 11.30%
Request 6: 35.52 ms | CPU 11.10% | Memory 11.30%
Request 7: 31.85 ms | CPU 0.00% | Memory 11.30%
Request 8: 31.96 ms | CPU 7.15% | Memory 11.30%
Request 9: 34.34 ms | CPU 7.15% | Memory 11.30%
Request 10: 34.16 ms | CPU 14